<a href="https://colab.research.google.com/github/ChoiSenn/MolTw-Not-X-/blob/main/MolTw_(Not_X).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from requests_oauthlib import OAuth1Session
import os
import json

# 개구졌는데 급하게 수정한 거라 그래 봐주셈 어쨌든 나중에 수정할거니까
consumer_key, consumer_secret, access_token, access_token_secret = "", "", "", ""

# --- API 인증 및 로그인 ------------------------------------------------------------------

def auth():
  consumer_key = "키들어갈곳"
  consumer_secret = "비밀키들어갈곳"

  request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
  oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

  try:
      fetch_response = oauth.fetch_request_token(request_token_url)
  except ValueError:
      print(
          "There may have been an issue with the consumer_key or consumer_secret you entered."
      )

  resource_owner_key = fetch_response.get("oauth_token")
  resource_owner_secret = fetch_response.get("oauth_token_secret")
  print("Got OAuth token: %s" % resource_owner_key)

  # Get authorization
  base_authorization_url = "https://api.twitter.com/oauth/authorize"
  authorization_url = oauth.authorization_url(base_authorization_url)
  print("Please go here and authorize: %s" % authorization_url)
  verifier = input("Paste the PIN here: ")

  # Get the access token
  access_token_url = "https://api.twitter.com/oauth/access_token"
  oauth = OAuth1Session(
      consumer_key,
      client_secret=consumer_secret,
      resource_owner_key=resource_owner_key,
      resource_owner_secret=resource_owner_secret,
      verifier=verifier,
  )
  oauth_tokens = oauth.fetch_access_token(access_token_url)

  access_token = oauth_tokens["oauth_token"]
  access_token_secret = oauth_tokens["oauth_token_secret"]

# --- 트윗 작성 ------------------------------------------------------------------

  while(True):
    tweet_text = input()
    if tweet_text == "exit":
      break

    payload = {"text": tweet_text}

    oauth = OAuth1Session(

        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )

    # Making the request
    response = oauth.post(
        "https://api.twitter.com/2/tweets",
        json=payload,
    )

    if response.status_code != 201:
        raise Exception(
            "Request returned an error: {} {}".format(response.status_code, response.text)
        )

    print("Response code: {}".format(response.status_code))

    # Saving the response as JSON
    json_response = response.json()
    print(json.dumps(json_response, indent=4, sort_keys=True))

# --- 메인 함수 ------------------------------------------------------------------

def main():
     auth()

if __name__ == "__main__":
    main()